In [1]:
import sqlite3
import zipfile

import pandas as pd

## Conhecer a base de dados dos acórdãos do TCU

A base de dados dos acórdãos do TCU foi disponibilizada para download público na plataforma Kaggle. Os dados dizem respeito aos acórdãos proferidos pelo TCU entre os anos de 1992 até 30/08/2019.
Acesse o endereço <https://www.kaggle.com/ferraz/acordaos-tcu> e baixe o arquivo no link Download (4GB) e salve na pasta '/dados'.
github: <https://github.com/netoferraz/acordaos-tcu>

In [ ]:
with zipfile.ZipFile('dados/330881_660826_bundle_archive.zip', 'r') as zip_ref:
    zip_ref.extractall()

In [3]:
# Cria conexão com a base de datos sqlite
conn = sqlite3.connect('dados/tcu-acordaos.db')

In [4]:
# Cria o dataframe df_acordaos
df_acordaos = pd.read_sql_query('SELECT * from acordaos', conn)

In [5]:
print(df_acordaos.shape, '\n')

# Mostra as colunas da base
print(df_acordaos.columns, '\n')

print(df_acordaos.info(), '\n')

(298942, 20) 

Index(['id', 'urn', 'ano_acordao', 'numero_acordao', 'relator', 'processo',
       'tipo_processo', 'data_sessao', 'numero_ata',
       'interessado_reponsavel_recorrente', 'entidade', 'representante_mp',
       'unidade_tecnica', 'repr_legal', 'assunto', 'sumario', 'acordao',
       'quorum', 'relatorio', 'voto'],
      dtype='object') 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298942 entries, 0 to 298941
Data columns (total 20 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   id                                 298942 non-null  int64 
 1   urn                                298942 non-null  object
 2   ano_acordao                        298942 non-null  int64 
 3   numero_acordao                     298123 non-null  object
 4   relator                            298470 non-null  object
 5   processo                           298562 non-null  object
 6   tipo_processo   

In [6]:
# Transfora em string as colunas ano_acordao
df_acordaos['ano_acordao'] = df_acordaos['ano_acordao'].map(str)

In [7]:
# Retira colunas que não interessam para o trabalho
df_acordaos = df_acordaos.drop(['representante_mp', 'assunto', 'unidade_tecnica', 'repr_legal', 'quorum',
                               'interessado_reponsavel_recorrente'], axis=1)

In [8]:
df_acordaos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 298942 entries, 0 to 298941
Data columns (total 14 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              298942 non-null  int64 
 1   urn             298942 non-null  object
 2   ano_acordao     298942 non-null  object
 3   numero_acordao  298123 non-null  object
 4   relator         298470 non-null  object
 5   processo        298562 non-null  object
 6   tipo_processo   298470 non-null  object
 7   data_sessao     298562 non-null  object
 8   numero_ata      298470 non-null  object
 9   entidade        297777 non-null  object
 10  sumario         269290 non-null  object
 11  acordao         298447 non-null  object
 12  relatorio       269199 non-null  object
 13  voto            269199 non-null  object
dtypes: int64(1), object(13)
memory usage: 31.9+ MB


In [9]:
print('Qtde de acordãos sem número de processo: ', df_acordaos[df_acordaos.processo.isnull()].shape[0])
print('Qtde de acordãos com tipo de processo SIGILOSO: ', df_acordaos[(df_acordaos.tipo_processo == 'SIGILOSO')].shape[0])
print('Qtde de acordãos com acordao SIGILOSO: ', df_acordaos[(df_acordaos.acordao == 'SIGILOSO') | (df_acordaos.acordao == 'None')].shape[0])
print('Qtde de acordãos com voto SIGILOSO: ', df_acordaos[(df_acordaos.voto == 'SIGILOSO')  | (df_acordaos.voto == 'None')].shape[0])
print('Qtde de acordãos com entidade SIGILOSO: ', df_acordaos[(df_acordaos.entidade == 'SIGILOSO')  | (df_acordaos.entidade == 'None')].shape[0])
print('Qtde de acordãos com Número de Acórdão não utilizado: ', df_acordaos[df_acordaos.sumario == 'Número de Acórdão não utilizado'].shape[0])

Qtde de acordãos sem número de processo:  380
Qtde de acordãos com tipo de processo SIGILOSO:  5201
Qtde de acordãos com acordao SIGILOSO:  1787
Qtde de acordãos com voto SIGILOSO:  2597
Qtde de acordãos com entidade SIGILOSO:  5542
Qtde de acordãos com Número de Acórdão não utilizado:  92


In [10]:
df_acordaos[(df_acordaos['entidade'] == 'SIGILOSO') | (df_acordaos['tipo_processo'].str.upper == 'SIGILOGO')
       | (df_acordaos['sumario'].str.upper == 'SIGILOGO')
       | (df_acordaos['acordao'].str.upper == 'SIGILOGO')
       | (df_acordaos['relatorio'].str.upper == 'SIGILOGO')
       | (df_acordaos['entidade'].str.upper == 'SIGILOGO')
       | (df_acordaos['voto'].str.upper == 'SIGILOGO')
        | (df_acordaos.sumario == 'Número de Acórdão não utilizado')].shape[0]

5630

In [11]:
df_acordaos = df_acordaos[df_acordaos.processo.notnull()]
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[df_acordaos['tipo_processo'] =='SIGILOSO'])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['acordao'] == 'SIGILOSO') | (df_acordaos['acordao'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['voto'] == 'SIGILOSO') | (df_acordaos['voto'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['entidade'] == 'SIGILOSO') | (df_acordaos['entidade'] == 'None')])
print(df_acordaos.shape[0])
df_acordaos = df_acordaos.drop(df_acordaos.index[(df_acordaos['sumario'] == 'Número de Acórdão não utilizado')])
print(df_acordaos.shape[0])

298562
293361
293087
292277
292207
292115


In [12]:
print('Qtde de acordãos sem número de processo: ', df_acordaos[df_acordaos.processo.isnull()].shape[0])
print('Qtde de acordãos com tipo de processo SIGILOSO: ', df_acordaos[(df_acordaos.tipo_processo == 'SIGILOSO')].shape[0])
print('Qtde de acordãos com acordao SIGILOSO: ', df_acordaos[(df_acordaos.acordao == 'SIGILOSO') | (df_acordaos.acordao == 'None')].shape[0])
print('Qtde de acordãos com voto SIGILOSO: ', df_acordaos[(df_acordaos.voto == 'SIGILOSO')  | (df_acordaos.voto == 'None')].shape[0])
print('Qtde de acordãos com entidade SIGILOSO: ', df_acordaos[(df_acordaos.entidade == 'SIGILOSO')  | (df_acordaos.entidade == 'None')].shape[0])
print('Qtde de acordãos com Número de Acórdão não utilizado: ', df_acordaos[df_acordaos.sumario == 'Número de Acórdão não utilizado'].shape[0])

Qtde de acordãos sem número de processo:  0
Qtde de acordãos com tipo de processo SIGILOSO:  0
Qtde de acordãos com acordao SIGILOSO:  0
Qtde de acordãos com voto SIGILOSO:  0
Qtde de acordãos com entidade SIGILOSO:  0
Qtde de acordãos com Número de Acórdão não utilizado:  0


In [13]:
df_acordaos[(df_acordaos['entidade'] == 'SIGILOSO') | (df_acordaos['tipo_processo'].str.upper == 'SIGILOGO')
       | (df_acordaos['sumario'].str.upper == 'SIGILOGO')
       | (df_acordaos['acordao'].str.upper == 'SIGILOGO')
       | (df_acordaos['relatorio'].str.upper == 'SIGILOGO')
       | (df_acordaos['voto'].str.upper == 'SIGILOGO')
        | (df_acordaos.sumario == 'Número de Acórdão não utilizado')].shape[0]

0

In [14]:
df_acordaos[df_acordaos.duplicated(['numero_acordao', 'processo', 'data_sessao'])]

,id,urn,ano_acordao,numero_acordao,relator,processo,tipo_processo,data_sessao,numero_ata,entidade,sumario,acordao,relatorio,voto
294848,294849,NA,2019,2011,AROLDO CEDRAZ,017.162/2007-1,Tomada de contas especial instaurada em razão ...,28/08/2019,33/2019-Plenário,Associação dos Irrigantes da Barragem de Terra...,TOMADA DE CONTAS ESPECIAL. NÃO COMPROVAÇÃO DA ...,"VISTOS, relatados e discutidos estes autos de ...","Inicio este Relatório transcrevendo, com algun...","Conforme consignado no Relatório precedente, t..."
295941,295942,NA,2019,2011,AROLDO CEDRAZ,017.162/2007-1,Tomada de contas especial instaurada em razão ...,28/08/2019,33/2019-Plenário,Associação dos Irrigantes da Barragem de Terra...,TOMADA DE CONTAS ESPECIAL. NÃO COMPROVAÇÃO DA ...,"VISTOS, relatados e discutidos estes autos de ...","Inicio este Relatório transcrevendo, com algun...","Conforme consignado no Relatório precedente, t..."


In [15]:
df_acordaos = df_acordaos.drop(295941)

In [ ]:
# Salva o arquivo em um csv compactado reduzindo o arquivo de 4GB para ~900MB
df_acordaos.to_csv('dados/df_acordaos.csv', sep='|', index=False, compression='gzip', encoding='utf-8')